In [1]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from web3 import Web3
import json
import time

# Load environment variables
load_dotenv()

# Connect to the Base network
w3 = Web3(Web3.HTTPProvider(os.getenv('RPC_URL')))

# Load contract ABIs
with open('AuctionHouse_ABI.json', 'r') as abi_file:
    auction_house_abi = json.load(abi_file)
    
with open('AuctionHouseFactory_ABI.json', 'r') as abi_file:
    auction_house_factory_abi = json.load(abi_file)
    
with open('AffiliateEscrow_ABI.json', 'r') as abi_file:
    affiliate_escrow_abi = json.load(abi_file)
    
    
with open('AuctionItemERC721_ABI.json', 'r') as abi_file:
    auction_item_erc721_abi = json.load(abi_file)
    
with open('AuctionItemERC721Factory_ABI.json', 'r') as abi_file:
    auction_item_erc721_factory_abi = json.load(abi_file)
    
with open('ERC20_ABI.json', 'r') as abi_file:
    erc20_abi = json.load(abi_file)

# Set up contract addresses
AUCTION_HOUSE_FACTORY_ADDRESS = os.getenv('AUCTION_HOUSE_FACTORY_ADDRESS')
AUCTION_ITEM_ERC721_FACTORY_ADDRESS = os.getenv('AUCTION_ITEM_ERC721_FACTORY_ADDRESS')

# Set up contract instances
auction_house_factory = w3.eth.contract(
    address=AUCTION_HOUSE_FACTORY_ADDRESS, 
    abi=auction_house_factory_abi
)


auction_item_erc721_factory = w3.eth.contract(
    address=AUCTION_ITEM_ERC721_FACTORY_ADDRESS, 
    abi=auction_item_erc721_factory_abi
)

# Set up accounts
owner_account = w3.eth.account.from_key(os.getenv('OWNER_PRIVATE_KEY'))
seller_account = w3.eth.account.from_key(os.getenv('SELLER_PRIVATE_KEY'))
bidder1_account = w3.eth.account.from_key(os.getenv('BIDDER1_PRIVATE_KEY'))
bidder2_account = w3.eth.account.from_key(os.getenv('BIDDER2_PRIVATE_KEY'))
affiliate_account = w3.eth.account.from_key(os.getenv('AFFILIATE_PRIVATE_KEY'))
arbiter_account = w3.eth.account.from_key(os.getenv('ARBITER_PRIVATE_KEY'))

# Example encrypted message structure for bids
EXAMPLE_MESSAGE = {
    "encryptedData": "0x",
    "ephemeralPublicKey": "0x",
    "iv": "0x",
    "verificationHash": "0x"
}

# Constants for the auction system
DEFAULT_SETTLEMENT_DEADLINE = 21 * 24 * 60 * 60  # 21 days in seconds

In [2]:
owner_account.address

'0x9f4640d04371ff6b7886ade5323746388107723a'

In [6]:
def create_auction_house(name, image, description, sender_account, custom_deadline=0):
    """Create a new auction house using the factory"""
    nonce = w3.eth.get_transaction_count(sender_account.address)
    
    # Get the addresses for required factories
    auction_item_factory_address = AUCTION_ITEM_ERC721_FACTORY_ADDRESS
    escrow_factory_address = os.getenv('AFFILIATE_ESCROW_FACTORY_ADDRESS')
    
    # Updated parameters to match the new contract interface
    txn = auction_house_factory.functions.createAuctionHouse(
        name,
        image,
        description,
        "https://example.com/contract-metadata",  # Contract URI
        "AUCTN",  # Symbol
        custom_deadline,
        auction_item_factory_address,
        escrow_factory_address
    ).build_transaction({
        'from': sender_account.address,
        'nonce': nonce,
        'gas': 69420000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = sender_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Transaction receipt: {tx_receipt}")
    
    # Process logs to get the new auction house address
    for log in tx_receipt['logs']:
        try:
            event = auction_house_factory.events.AuctionHouseCreated().process_log(log)
            if event:
                return event['args']['auctionHouse']
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No AuctionHouseCreated event found in the transaction logs")
    return None

def create_auction_with_new_nft(auction_house_contract, nft_contract_address, nft_metadata, 
                               start_time, reserve_price, duration, affiliate_fee, arbiter_address,
                               payment_token=None, premium=False, premium_rate=0, 
                               min_bid_increment=500, time_extension=900):
    """Create a new auction with a new NFT in one transaction"""
    if payment_token is None:
        payment_token = "0x0000000000000000000000000000000000000000"  # ETH
    
    # Get the escrow factory address from the auction house
    escrow_factory_address = auction_house_contract.functions.escrowFactory().call()
    
    nonce = w3.eth.get_transaction_count(seller_account.address)
    
    # Prepare the NFT metadata for the contract
    contract_metadata = {
        "name": nft_metadata["name"],
        "description": nft_metadata["description"],
        "image": nft_metadata["image"],
        "termsOfService": nft_metadata.get("termsOfService", ""),
        "supplementalImages": nft_metadata.get("supplementalImages", [])
    }
    
    # Build the transaction
    txn = auction_house_contract.functions.createAuctionWithNewNFT(
        nft_contract_address,
        [
            contract_metadata["name"],
            contract_metadata["description"],
            contract_metadata["image"],
            contract_metadata["termsOfService"],
            contract_metadata["supplementalImages"]
        ],
        start_time,
        reserve_price,
        duration,
        affiliate_fee,
        arbiter_address,
        escrow_factory_address,
        payment_token,
        premium,
        premium_rate,
        min_bid_increment,
        time_extension
    ).build_transaction({
        'from': seller_account.address,
        'nonce': nonce,
        'gas': 6000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = seller_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Process logs to get the auction ID
    auction_id = None
    for log in tx_receipt['logs']:
        try:
            event = auction_house_contract.events.AuctionCreated().process_log(log)
            if event:
                auction_id = event['args']['auctionId']
                break
        except Exception as e:
            continue
    
    if auction_id is None:
        print("No AuctionCreated event found in the transaction logs")
        return None
    
    print(f"Created auction with ID {auction_id} and minted a new NFT")
    
    # Get the auction data to find the token ID
    auction_data = auction_house_contract.functions.auctions(auction_id).call()
    token_id = auction_data[1]  # Index of tokenId in the Auction struct
    
    print(f"NFT minted with token ID: {token_id}")
    
    return {
        "auction_id": auction_id,
        "token_id": token_id,
        "tx_receipt": tx_receipt
    }

# Example usage:
# current_time = int(time.time())
# start_time = current_time + 300  # Start in 5 minutes
# reserve_price = w3.to_wei(0.01, 'ether')  # 0.01 ETH
# duration = 24 * 60 * 60  # 24 hours
# affiliate_fee = 500  # 5%
# 
# nft_metadata = {
#     "name": "Auction Item #1",
#     "description": "This is a unique digital item being auctioned",
#     "image": "https://example.com/nft-image.png",
#     "termsOfService": "By bidding on this item, you agree to the terms of service.",
#     "supplementalImages": ["https://example.com/supplemental1.png", "https://example.com/supplemental2.png"]
# }
# 
# result = create_auction_with_new_nft(
#     auction_house,
#     nft_contract_address,
#     nft_metadata,
#     start_time,
#     reserve_price,
#     duration,
#     affiliate_fee,
#     arbiter_account.address
# )
# Add a function to create an NFT contract through the auction house
def create_nft_contract_via_auction_house(auction_house_contract, name, symbol, contract_uri):
    """Create a new NFT contract through the auction house"""
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = auction_house_contract.functions.createNFTContract(
        name,
        symbol,
        contract_uri
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': int(1.1 * w3.eth.gas_price),
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Get the NFT contract address from the mapping
    nft_contract_address = auction_house_contract.functions.nftContracts(symbol).call()
    print(f"Created new NFT contract via auction house at: {nft_contract_address}")
    return nft_contract_address

# Update the mint_auction_item function to work with the auction house
def mint_auction_item(token_address, metadata, to_address=None):
    """Mint a new token with metadata and return its ID"""
    # Create contract instance
    token_contract = w3.eth.contract(
        address=token_address,
        abi=auction_item_erc721_abi
    )
    
    # Use provided address or default to owner account
    mint_to_address = to_address if to_address else owner_account.address
    
    # Mint token to the specified address
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = token_contract.functions.mintWithMetadata(
        mint_to_address,
        metadata["name"],
        metadata["description"],
        metadata["image"],
        metadata["termsOfService"],
        metadata.get("supplementalImages", [])
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': int(1.1 * w3.eth.gas_price),
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Get the token ID from the event
    token_id = None
    for log in tx_receipt['logs']:
        try:
            event = token_contract.events.Transfer().process_log(log)
            if event and event['args']['from'] == '0x0000000000000000000000000000000000000000':
                token_id = event['args']['tokenId']
                print(f"Minted token ID: {token_id} to address: {mint_to_address}")
                break
        except Exception as e:
            continue
    
    if token_id is None:
        print("Failed to get token ID from mint transaction")
        return None
    
    return token_id

# Add a function to approve the auction house to transfer the NFT
def approve_nft_for_auction(token_address, token_id, auction_house_address):
    """Approve the auction house to transfer the NFT"""
    token_contract = w3.eth.contract(
        address=token_address,
        abi=auction_item_erc721_abi
    )
    
    nonce = w3.eth.get_transaction_count(seller_account.address)
    
    txn = token_contract.functions.approve(
        auction_house_address,
        token_id
    ).build_transaction({
        'from': seller_account.address,
        'nonce': nonce,
        'gas': 200000,
        'gasPrice': int(1.1 * w3.eth.gas_price),
    })
    
    signed_txn = seller_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Approved auction house {auction_house_address} to transfer NFT {token_id}")
    return tx_receipt

# Update the create_auction function to work with the new parameters
def create_auction(auction_house_contract, token_address, token_id, start_time, reserve_price, 
                  duration, affiliate_fee, arbiter_address, 
                  payment_token=None, premium=False, premium_rate=0, 
                  min_bid_increment=500, time_extension=900):
    """Create a new auction"""
    if payment_token is None:
        payment_token = "0x0000000000000000000000000000000000000000"  # ETH
    
    # Get the escrow factory address from the auction house
    escrow_factory_address = auction_house_contract.functions.escrowFactory().call()
    
    nonce = w3.eth.get_transaction_count(seller_account.address)
    
    txn = auction_house_contract.functions.createAuction(
        token_address,
        token_id,
        start_time,
        reserve_price,
        duration,
        affiliate_fee,
        arbiter_address,
        escrow_factory_address,
        payment_token,
        premium,
        premium_rate,
        min_bid_increment,
        time_extension
    ).build_transaction({
        'from': seller_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = seller_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Process logs to get the auction ID
    for log in tx_receipt['logs']:
        try:
            event = auction_house_contract.events.AuctionCreated().process_log(log)
            if event:
                return event['args']['auctionId']
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No AuctionCreated event found in the transaction logs")
    return None

# Add a function to settle the escrow after auction ends
def settle_escrow(escrow_address, token_address=None):
    """Settle the escrow after auction ends"""
    # Create escrow contract instance
    escrow_contract = w3.eth.contract(
        address=escrow_address,
        abi=affiliate_escrow_abi
    )
    
    # Get the payer (winning bidder)
    payer = escrow_contract.functions.payer().call()
    
    # Create a transaction to settle the escrow
    # Use the bidder account that matches the payer
    if payer == bidder1_account.address:
        bidder_account = bidder1_account
    elif payer == bidder2_account.address:
        bidder_account = bidder2_account
    else:
        print(f"Unknown payer address: {payer}")
        return None
    
    nonce = w3.eth.get_transaction_count(bidder_account.address)
    
    # Get the escrow balance
    if token_address is None or token_address == "0x0000000000000000000000000000000000000000":
        token_address = "0x0000000000000000000000000000000000000000"  # ETH
        escrow_balance = w3.eth.get_balance(escrow_address)
    else:
        token_contract = w3.eth.contract(address=token_address, abi=erc20_abi)
        escrow_balance = token_contract.functions.balanceOf(escrow_address).call()
    
    txn = escrow_contract.functions.settle(
        token_address,
        escrow_balance
    ).build_transaction({
        'from': bidder_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': int(1.1 * w3.eth.gas_price),
    })
    
    signed_txn = bidder_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Settled escrow at {escrow_address}")
    return tx_receipt

In [4]:
AUCTION_HOUSE_FACTORY_ADDRESS

'0xe7C121cb8773d324b68d1fb3531Fc9043440D1e0'

In [7]:
# Step 1: Create a new auction house
auction_house_address = create_auction_house(
    "Test Auction House",
    "https://example.com/image.png",
    "A test auction house for NFTs",
    owner_account,  # Added the sender account parameter
    30 * 24 * 60 * 60  # 30 days settlement deadline
)

print(f"New auction house created at: {auction_house_address}")

# Create auction house contract instance
auction_house = w3.eth.contract(
    address=auction_house_address,
    abi=auction_house_abi
)

Transaction receipt: AttributeDict({'blockHash': HexBytes('0xcd06a5efdfc3f965402df1e90916ccad14182e477bef1217cf89129b8702dc3b'), 'blockNumber': 29842730, 'contractAddress': None, 'cumulativeGasUsed': 42637878, 'effectiveGasPrice': 1410153, 'from': '0x9f4640d04371ff6b7886ade5323746388107723a', 'gasUsed': 5813499, 'l1BaseFeeScalar': '0x8dd', 'l1BlobBaseFee': '0x1ddf86', 'l1BlobBaseFeeScalar': '0x101c12', 'l1Fee': '0x105770702', 'l1GasPrice': '0x1933ab2a', 'l1GasUsed': '0xfbd', 'logs': [AttributeDict({'address': '0xcc258f97de13BD0a1214D4D33b84D95862fe8B58', 'blockHash': HexBytes('0xcd06a5efdfc3f965402df1e90916ccad14182e477bef1217cf89129b8702dc3b'), 'blockNumber': 29842730, 'blockTimestamp': '0x68191737', 'data': HexBytes('0x'), 'logIndex': 604, 'removed': False, 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x000000000000000000000000e7c121cb8773d324b68d1fb

In [25]:
# Complete test flow for creating an auction with a new NFT
# Step 1: Set up parameters
current_time = int(time.time())
start_time = current_time + 60  # Start in 1 minute
reserve_price = w3.to_wei(0.01, 'ether')  # 0.01 ETH
duration = 5 * 60  # 5 minutes for testing
affiliate_fee = 500  # 5%

nft_metadata = {
    "name": "Auction Item #1",
    "description": "This is a unique digital item being auctioned",
    "image": "https://example.com/nft-image.png",
    "termsOfService": "By bidding on this item, you agree to the terms of service.",
    "supplementalImages": ["https://example.com/supplemental1.png", "https://example.com/supplemental2.png"]
}

# Step 2: Create the auction with a new NFT
print("Creating auction with new NFT...")
result = create_auction_with_new_nft(
    auction_house,
    nft_contract_address,
    nft_metadata,
    start_time,
    reserve_price,
    duration,
    affiliate_fee,
    arbiter_account.address
)

auction_id = result["auction_id"]
token_id = result["token_id"]

print(f"Created auction with ID {auction_id} for NFT with token ID {token_id}")


ABIFunctionNotFound: ("The function 'createNFTContract' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')

In [ ]:

# Step 3: Wait for the auction to start
print(f"Waiting for auction to start at {start_time} (current time: {current_time})")
time.sleep(max(0, start_time - current_time) + 5)  # Add 5 seconds buffer

# Step 4: Place bids
min_bid = get_minimum_bid(auction_house, auction_id)
bid1_amount = min_bid + w3.to_wei(0.001, 'ether')  # Add 0.001 ETH to minimum

print(f"Bidder 1 placing bid of {w3.from_wei(bid1_amount, 'ether')} ETH")
place_bid(auction_house, auction_id, bidder1_account, affiliate_account.address, EXAMPLE_MESSAGE, bid1_amount)

# Get the new minimum bid
min_bid = get_minimum_bid(auction_house, auction_id)
bid2_amount = min_bid + w3.to_wei(0.002, 'ether')  # Add 0.002 ETH to minimum

print(f"Bidder 2 placing bid of {w3.from_wei(bid2_amount, 'ether')} ETH")
place_bid(auction_house, auction_id, bidder2_account, affiliate_account.address, EXAMPLE_MESSAGE, bid2_amount)

# Step 5: Wait for the auction to end
auction_end_time = start_time + duration
current_time = int(time.time())
wait_time = max(0, auction_end_time - current_time) + 5  # Add 5 seconds buffer
print(f"Waiting for auction to end at {auction_end_time} (current time: {current_time}, waiting {wait_time} seconds)")
time.sleep(wait_time)

# Step 6: End the auction
print("Ending the auction...")
end_tx = end_auction(auction_house, auction_id)

# Step 7: Get the escrow address from the auction
auction_details = auction_house.functions.auctions(auction_id).call()
escrow_address = auction_details[11]  # Index of escrowAddress in the Auction struct
print(f"Escrow address: {escrow_address}")

# Step 8: Settle the escrow
print("Settling the escrow...")
settle_tx = settle_escrow(escrow_address)

# Step 9: Verify the NFT ownership
winner_address = auction_details[8]  # Index of bidder in the Auction struct
nft_contract = w3.eth.contract(address=nft_contract_address, abi=auction_item_erc721_abi)
current_owner = nft_contract.functions.ownerOf(token_id).call()
print(f"Winner address: {winner_address}")
print(f"Current NFT owner: {current_owner}")

if current_owner.lower() == winner_address.lower():
    print("✅ Success! NFT transferred to the winning bidder")
else:
    print("❌ Error: NFT not transferred to the winning bidder")

In [8]:
# Step 2: Create an NFT contract for multiple auctions
x = create_auction_item_erc721(
    "Test NFT",
    "TEST",
    "https://example.com/nft.json",
    owner_account  # Add the sender account parameter
)

print(f"New NFT contract created at: {x}")

Transaction receipt logs:
Log 0: AttributeDict({'address': '0xC9b577fE25018CBc12fcee1650c3551Aca21B2F9', 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x0000000000000000000000006e1e335f48cde44119361e2cfddda40ed3bd41cb')], 'data': HexBytes('0x'), 'blockNumber': 29534012, 'transactionHash': HexBytes('0x36c88ca95b94b08389e2add8039a7e8bc36773aeb7fed6917fa0c507c07c7c07'), 'transactionIndex': 190, 'blockHash': HexBytes('0x705bb0578314d7e7b2f59961660b0894ecfa6238a47116631a273e1222d20dd3'), 'logIndex': 508, 'removed': False})
Log 1: AttributeDict({'address': '0xC9b577fE25018CBc12fcee1650c3551Aca21B2F9', 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000006e1e335f48cde44119361e2cfddda40ed3bd41cb'), HexBytes('0x0000000000000000000000009f4640d04371ff6b7886ade5323746388107723a')], 'data': He

In [13]:

# Step 3: Mint an NFT to the seller
nft_metadata = {
    "name": "Test NFT #1",
    "description": "A test NFT for auction",
    "image": "https://example.com/nft1.png",
    "termsOfService": "https://example.com/terms",
    "supplementalImages": ["https://example.com/nft1-detail.png"]
}

# Create NFT contract instance
nft_contract = w3.eth.contract(
    address=nft_contract_address,
    abi=auction_item_erc721_abi
)

# Mint the NFT to the seller
token_id = mint_auction_item(nft_contract_address, nft_metadata, seller_account.address)
print(f"Minted NFT with ID {token_id} to seller {seller_account.address}")

# Step 4: Approve the auction house to transfer the NFT
approve_nft_for_auction(nft_contract_address, token_id, auction_house_address)

# Step 5: Create an auction with a short duration (5 minutes)
current_time = int(time.time())
start_time = current_time + 60  # Start in 1 minute
duration = 5 * 60  # 5 minutes
reserve_price = w3.to_wei(0.1, 'ether')  # 0.1 ETH
affiliate_fee = 500  # 5%

auction_id = create_auction(
    auction_house,
    nft_contract_address,
    token_id,
    start_time,
    reserve_price,
    duration,
    affiliate_fee,
    arbiter_account.address
)

print(f"Created auction with ID {auction_id}")

# Step 6: Wait for the auction to start
print(f"Waiting for auction to start at {start_time} (current time: {current_time})")
time.sleep(max(0, start_time - current_time) + 5)  # Add 5 seconds buffer

# Step 7: Place bids
min_bid = get_minimum_bid(auction_house, auction_id)
bid1_amount = min_bid + w3.to_wei(0.01, 'ether')  # Add 0.01 ETH to minimum

print(f"Bidder 1 placing bid of {w3.from_wei(bid1_amount, 'ether')} ETH")
place_bid(auction_house, auction_id, bidder1_account, affiliate_account.address, EXAMPLE_MESSAGE, bid1_amount)

# Get the new minimum bid
min_bid = get_minimum_bid(auction_house, auction_id)
bid2_amount = min_bid + w3.to_wei(0.02, 'ether')  # Add 0.02 ETH to minimum

print(f"Bidder 2 placing bid of {w3.from_wei(bid2_amount, 'ether')} ETH")
place_bid(auction_house, auction_id, bidder2_account, affiliate_account.address, EXAMPLE_MESSAGE, bid2_amount)

# Step 8: Wait for the auction to end
auction_end_time = start_time + duration
current_time = int(time.time())
wait_time = max(0, auction_end_time - current_time) + 5  # Add 5 seconds buffer
print(f"Waiting for auction to end at {auction_end_time} (current time: {current_time}, waiting {wait_time} seconds)")
time.sleep(wait_time)

# Step 9: End the auction
print("Ending the auction...")
end_tx = end_auction(auction_house, auction_id)

# Step 10: Get the escrow address from the auction
auction_details = auction_house.functions.auctions(auction_id).call()
escrow_address = auction_details[11]  # Index of escrowAddress in the Auction struct
print(f"Escrow address: {escrow_address}")

# Step 11: Settle the escrow
print("Settling the escrow...")
settle_tx = settle_escrow(escrow_address)

# Step 12: Verify the NFT ownership
winner_address = auction_details[8]  # Index of bidder in the Auction struct
current_owner = nft_contract.functions.ownerOf(token_id).call()
print(f"Winner address: {winner_address}")
print(f"Current NFT owner: {current_owner}")

if current_owner.lower() == winner_address.lower():
    print("✅ Success! NFT transferred to the winning bidder")
else:
    print("❌ Error: NFT not transferred to the winning bidder")

'0xc9b577fe25018cbc12fcee1650c3551aca21b2f9'

In [11]:
# Step 3: Mint the item to the auction house
token_id = mint_auction_item(
    x,  # NFT contract address
    {
        "name": "Test Item", 
        "description": "A test item", 
        "image": "https://example.com/nft.png", 
        "termsOfService": "https://example.com/terms.txt"
    },
    owner_account,  # Add the sender account parameter
    auction_house_address  # Optional: directly mint to the auction house
)

print(f"Minted token with ID: {token_id}")

Failed to get token ID from mint transaction
Minted token with ID: None


In [12]:
owner_account.address

'0x9f4640d04371ff6b7886ade5323746388107723a'

In [3]:
# Create an auction with affiliate fee set to 100
auction_id = create_auction(
    auction_house,  # Use the contract instance, not the address
    x,  # NFT contract address
    1,  # Token ID
    int(time.time()),  # Start time (now)
    w3.to_wei(0.0001, 'ether'),  # Reserve price
    7 * 24 * 60 * 60,  # Duration (7 days in seconds)
    100,  # Affiliate fee
    arbiter_account.address,  # Arbiter address
    os.getenv('AFFILIATE_ESCROW_FACTORY_ADDRESS'),  # Escrow factory address
    seller_account  # Sender account (the seller)
)

print(f"Created auction with ID: {auction_id}")

NameError: name 'auction_house' is not defined

In [7]:
#load a list of transactions from lw_3171415_3399531.txt
with open('lw_3171415_3399531.txt', 'r') as file:
    transactions = file.readlines()

#for each transaction, place a bid
for transaction in transactions:
    place_bid(auction_house, auction_id, bidder_account, affiliate_address, message, bid_amount)
